In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

import torch
import torch.nn as nn
from torch.functional import F
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.data import Data, HeteroData
from torch_geometric.loader import DataListLoader, DataLoader
from torch_geometric.utils.convert import to_networkx
import networkx as nx

from sklearn.preprocessing import MinMaxScaler

In [23]:
# Load network data: node features, edge_features and edge_index
node_features = pd.read_csv('network/node_features.csv', index_col=0, header=0).astype(float).fillna(value=0)
branch_index = pd.read_csv('network/branch_index.csv', index_col=0, header=0)
trafo_index = pd.read_csv('network/trafo_index.csv', index_col=0, header=0)
branch_attr = pd.read_csv('network/branch_attr.csv', index_col=0, header=0)
trafo_attr = pd.read_csv('network/trafo_attr.csv', index_col=0, header=0)



Constructed style path: /Users/lylakiani/Desktop/Research/power_system_project/plot_style.mplstyle


OSError: '/Users/lylakiani/Desktop/Research/power_system_project/plot_style.mplstyle' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

In [20]:

%config InlineBackend.figure_format = 'png2x'
plt.style.use('/absolute/path/to/plot_style.mplstyle')


# %%
random.seed(10)
np.random.seed(20)

# %%
# Load network data: node features, edge_features and edge_index
node_features = pd.read_csv('network/node_features.csv', index_col=0, header=0)
node_features = node_features.astype(np.float64).fillna(value=0)
branch_index = pd.read_csv('network/branch_index.csv', index_col=0, header=0)
trafo_index = pd.read_csv('network/trafo_index.csv', index_col=0, header=0)
branch_attr = pd.read_csv('network/branch_attr.csv', index_col=0, header=0)
trafo_attr = pd.read_csv('network/trafo_attr.csv', index_col=0, header=0)

# %%
node_features.shape, branch_index.shape, trafo_index.shape, branch_attr.shape, trafo_attr.shape

# %%
branch_index = torch.tensor(branch_index.to_numpy().T, dtype=torch.long)
trafo_index = torch.tensor(trafo_index.to_numpy().T, dtype=torch.long)

# %%
# Load synthetic 'p_mw' at load buses
p_load_data = pd.read_csv('synthetic_data/p_load_data.csv', index_col=0, header=0)
p_load_data.shape

# %%
# Load synthetic 'p_mw' at generator buses
p_gen_data = pd.read_csv('synthetic_data/p_gen_data.csv', index_col=0, header=0)
p_gen_data.shape

# %%
# Load scaled 'p_mw' at all buses
p_bus_data = pd.read_csv('synthetic_data/p_bus_data.csv', index_col=0, header=0)
p_bus_data.shape

# %%
p_slack_max_data = pd.read_csv('synthetic_data/p_slack_max_data.csv', index_col=0, header=0)
p_slack_max_data.shape

# %%
## All bus
bus = np.arange(node_features.shape[0])

## All gen bus
gen_bus = pd.read_csv('zones/gen_bus.csv', 
                      index_col=None, 
                      header=None).to_numpy()
gen_bus = gen_bus.reshape((-1)) - 1
# Gen bus mask
gen_no = np.isin(bus, gen_bus)

# All load bus
load_bus = pd.read_csv('zones/load_bus.csv', 
                       index_col=None, 
                       header=None).to_numpy()
load_bus = load_bus.reshape((-1)) - 1
# Load bus mask
load_no = np.isin(bus, load_bus)

# Wind bus
re_gen_bus = pd.read_csv('zones/wind_bus.csv', 
                         index_col=None, 
                         header=None).to_numpy()
re_gen_bus = re_gen_bus.reshape((-1)) - 1
# Wind bus mask
re_gen_no = np.isin(gen_bus, re_gen_bus)
not_re_gen_no = ~re_gen_no

# %%
re_gen_bus.shape, re_gen_no.shape, not_re_gen_no.shape, gen_no.shape, load_no.shape

# %% [markdown]
# #### Node prediction dataset

# %%
# Store multiple Data() in a list
data_list = []

for i in range(p_load_data.shape[1]):
    node_features['load_p_mw'][load_no] = p_load_data.iloc[:, i].values
    node_features['max_gen_p_mw'][re_gen_bus] = p_gen_data.iloc[re_gen_no, i].values
    node_features['min_gen_p_mw'][re_gen_bus] = p_gen_data.iloc[re_gen_no, i].values
    node_features['max_gen_p_mw'][271] = p_slack_max_data.iloc[i].values

    X = node_features.to_numpy()
    y = p_bus_data.iloc[:, i].to_numpy()
    X = torch.from_numpy(X).float()
    y = torch.from_numpy(y.reshape((-1))).float()

    data = HeteroData()
    data['node'].x = X
    data['node'].y = y

    # Set edge index
    data['node', 'branch', 'node'].edge_index = branch_index
    data['node', 'trafo', 'node'].edge_index = trafo_index
    data['node', 'branch', 'node'].edge_attr = torch.from_numpy(branch_attr.to_numpy()).float()
    data['node', 'trafo', 'node'].edge_attr = torch.from_numpy(trafo_attr.to_numpy()).float()
    
    data_list.append(data)

# %%
## Save train and test dataset
train_size = int(0.7*len(data_list))
val_size = int(0.1*len(data_list))
test_size = int(0.2*len(data_list))

# Create PyTorch Geometric DataLoader()
batch_size = 1
train_data_loader = DataLoader(dataset=data_list[:train_size], batch_size=batch_size, shuffle=False)
val_data_loader = DataLoader(dataset=data_list[train_size:train_size+val_size], batch_size=batch_size, shuffle=False)
test_data_loader = DataLoader(dataset=data_list[-test_size:], batch_size=batch_size, shuffle=False)

# Save train and test datasets
torch.save(train_data_loader, 'train_test_dataset/node_prediction_train_dataset.pt')
torch.save(val_data_loader, 'train_test_dataset/node_prediction_val_dataset.pt')
torch.save(test_data_loader, 'train_test_dataset/node_prediction_test_dataset.pt')




OSError: '/absolute/path/to/plot_style.mplstyle' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)